<a href="https://colab.research.google.com/github/tusharpamnani/DL-Lab-Mini-Project/blob/main/Credit_Card_Fraud_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly Detection in Credit Card Transactions using Deep Learning

## Objective
Detect fraudulent credit card transactions using Autoencoders and LSTM-based models.

**Dataset:** [Kaggle - Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud)


In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed


In [2]:
# Load Data
df = pd.read_csv('/content/creditcard.csv')  # Ensure the dataset is downloaded and placed in the same directory
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
# Preprocessing
df['Amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1, 1))
df = df.drop(['Time'], axis=1)
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)


In [4]:
# Autoencoder Model
input_dim = X_train.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(14, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train, X_train, epochs=10, batch_size=256, shuffle=True, validation_data=(X_test, X_test))

Epoch 1/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.1137 - val_loss: 0.8723
Epoch 2/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8863 - val_loss: 0.8329
Epoch 3/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.8469 - val_loss: 0.8173
Epoch 4/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8577 - val_loss: 0.8060
Epoch 5/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.8480 - val_loss: 0.7988
Epoch 6/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8229 - val_loss: 0.7947
Epoch 7/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.8066 - val_loss: 0.7922
Epoch 8/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.8236 - val_loss: 0.7905
Epoch 9/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7952 - val_loss: 0.7894
Epoch 10/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.7900 - val_loss: 0.7888


In [5]:
# Anomaly Detection using Reconstruction Error
reconstructions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructions, 2), axis=1)
threshold = np.percentile(mse, 95)
y_pred = (mse > threshold).astype(int)
print(classification_report(y[y.index.isin(X_test.index)], y_pred))

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     56864
           1       0.00      0.03      0.00        98

    accuracy                           0.95     56962
   macro avg       0.50      0.49      0.49     56962
weighted avg       1.00      0.95      0.97     56962



## LSTM Autoencoder Model
Using sequence modeling to detect temporal anomalies in transactions.

In [6]:
# Reshape for LSTM
X_train_lstm = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# LSTM Autoencoder
inputs = Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))
encoded = LSTM(32, activation='relu', return_sequences=False)(inputs)
repeat = RepeatVector(X_train_lstm.shape[1])(encoded)
decoded = LSTM(X_train_lstm.shape[2], activation='relu', return_sequences=True)(repeat)
decoded = TimeDistributed(Dense(X_train_lstm.shape[2]))(decoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(optimizer='adam', loss='mse')
lstm_autoencoder.fit(X_train_lstm, X_train_lstm, epochs=10, batch_size=128, validation_data=(X_test_lstm, X_test_lstm))

Epoch 1/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.4819 - val_loss: 0.0461
Epoch 2/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.0363 - val_loss: 0.0118
Epoch 3/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.0120 - val_loss: 0.0070
Epoch 4/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 0.0071 - val_loss: 0.0041
Epoch 5/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 0.0049 - val_loss: 0.0033
Epoch 6/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0034 - val_loss: 0.0031
Epoch 7/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0035 - val_loss: 0.0015
Epoch 8/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0020 - val_loss: 0.0198
Epoch 9/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.0056 - val_loss: 9.7626e-04
Epoch 10/10
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.0010 - val_loss: 8.3447e-04


In [7]:
# LSTM Reconstruction Error
reconstructions = lstm_autoencoder.predict(X_test_lstm)
mse = np.mean(np.power(X_test_lstm - reconstructions, 2), axis=(1, 2))
threshold = np.percentile(mse, 95)
y_pred_lstm = (mse > threshold).astype(int)
print("LSTM Autoencoder")
print(classification_report(y[y.index.isin(X_test.index)], y_pred_lstm))

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
LSTM Autoencoder
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     56864
           1       0.00      0.06      0.00        98

    accuracy                           0.95     56962
   macro avg       0.50      0.51      0.49     56962
weighted avg       1.00      0.95      0.97     56962



## Feedforward Neural Network (FFNN) Classifier
A supervised classification model to benchmark against unsupervised models.

In [8]:
# FFNN Model
from tensorflow.keras.utils import to_categorical
model = tf.keras.Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y[y.index.isin(X_train.index)], epochs=10, batch_size=256, validation_data=(X_test, y[y.index.isin(X_test.index)]))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9259 - loss: 0.1912 - val_accuracy: 0.9983 - val_loss: 0.0138
Epoch 2/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9981 - loss: 0.0146 - val_accuracy: 0.9983 - val_loss: 0.0137
Epoch 3/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9984 - loss: 0.0126 - val_accuracy: 0.9983 - val_loss: 0.0135
Epoch 4/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9983 - loss: 0.0133 - val_accuracy: 0.9983 - val_loss: 0.0134
Epoch 5/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9984 - loss: 0.0120 - val_accuracy: 0.9983 - val_loss: 0.0133
Epoch 6/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9982 - loss: 0.0132 - val_accuracy: 0.9983 - val_loss: 0.0132
Epoch 7/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9983 - loss: 0.0127 - val_accuracy: 0.9983 - val_loss: 0.0132
Epoch 8/10
891/891 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9982 - loss: 0.0129 - val_accuracy: 0.

In [9]:
# Evaluation for FFNN
y_pred_ffnn = (model.predict(X_test) > 0.5).astype(int)
print("Feedforward Neural Network")
print(classification_report(y[y.index.isin(X_test.index)], y_pred_ffnn))

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Feedforward Neural Network
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.00      0.00      0.00        98

    accuracy                           1.00     56962
   macro avg       0.50      0.50      0.50     56962
weighted avg       1.00      1.00      1.00     56962



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
